In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import keras

from tensorflow.keras import layers
from tensorflow.keras import losses

2024-05-25 11:10:15.973242: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-25 11:10:15.976702: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-25 11:10:17.097510: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [139]:
def custom_standardization(input_data):
  s = input_data.lower()
  s = re.sub(r'http\S+', ' ', s)
  s = re.sub(r'@\S+', '', s)
  s = re.sub(r'[\W\d]', ' ', s)
  return re.sub(r'\s+', ' ', s)


In [185]:
data_pos = pd.read_csv("/home/alex/Data/datasets/RuTweetCorp/positive.csv")
data_neg = pd.read_csv("/home/alex/Data/datasets/RuTweetCorp/negative.csv")

In [186]:
data_pos.head()
data_neg.head()

,id,tdate,tname,ttext,ttype,trep,trtw,tfav,tstcount,tfoll,tfrien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [187]:
raw_text_pos = data_pos.pop("ttext")
raw_text_neg = data_neg.pop("ttext")

In [188]:
raw_text_pos = raw_text_pos.map(custom_standardization)
raw_text_neg = raw_text_neg.map(custom_standardization)

In [208]:
raw_text_pos[101]

' единственное что меня расстраивает на бали невозможность сходить в кинотеатр с друзьями на интересный фильм '

In [190]:
raw_text_np_pos = np.array(raw_text_pos)
raw_text_np_neg = np.array(raw_text_neg)

In [191]:
pos_labels = np.empty(len(raw_text_np_pos))
neg_labels = np.empty(len(raw_text_np_neg))
pos_labels[:] = 1
neg_labels[:] = 0

In [192]:
labels = np.concatenate((pos_labels, neg_labels), axis=0)

In [193]:
raw_text_np = np.concatenate((raw_text_np_pos, raw_text_np_neg), axis=0)

In [194]:
max_features = 10000
sequence_length = 100

vectorize_layer = layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [195]:

vectorize_layer.adapt(raw_text_np)

In [196]:
print("1289 ---> ",vectorize_layer.get_vocabulary()[1289])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

1289 --->  айфон
 313 --->  иногда
Vocabulary size: 10000


In [143]:
vectorize_layer.get_vocabulary()


['',
 '[UNK]',
 'не',
 'я',
 'и',
 'в',
 'на',
 'rt',
 'а',
 'что',
 't',
 'http',
 'co',
 'с',
 'как',
 'у',
 'меня',
 'то',
 'это',
 'так',
 'мне',
 'd',
 'все',
 'но',
 'ты',
 'ну',
 'по',
 'за',
 'уже',
 'вот',
 'да',
 'же',
 'только',
 'сегодня',
 'еще',
 'бы',
 'нет',
 'когда',
 'очень',
 'хочу',
 'тебя',
 'к',
 'из',
 'он',
 'день',
 'просто',
 'будет',
 'мы',
 'от',
 'тебе',
 'было',
 'если',
 'о',
 'теперь',
 'надо',
 'даже',
 'завтра',
 'тоже',
 'кто',
 'до',
 'там',
 'есть',
 'его',
 'вообще',
 'всё',
 'она',
 'для',
 'спасибо',
 'сейчас',
 'нас',
 'буду',
 'почему',
 'могу',
 'блин',
 'люблю',
 'вы',
 'без',
 'они',
 'знаю',
 'тут',
 'или',
 'раз',
 'мой',
 'ничего',
 'больше',
 'со',
 'чем',
 'всегда',
 'хорошо',
 'про',
 'можно',
 'всем',
 'может',
 'дома',
 'год',
 'спать',
 'потом',
 'был',
 'где',
 'ещё',
 'время',
 'такой',
 'новый',
 'их',
 'ни',
 'опять',
 'после',
 'этот',
 'чтобы',
 'моя',
 'делать',
 'была',
 'пока',
 'какой',
 'себя',
 'быть',
 'вас',
 'всех',
 

In [197]:
text_np = vectorize_layer(raw_text_np)

In [198]:
text_np

<tf.Tensor: shape=(226834, 100), dtype=int64, numpy=
array([[ 117,    3,    4, ...,    0,    0,    0],
       [  27,   19,  181, ...,    0,    0,    0],
       [   7,   22,   21, ...,    0,    0,    0],
       ...,
       [  26,    4,    5, ...,    0,    0,    0],
       [   7,    1,    2, ...,    0,    0,    0],
       [2757, 1077,   13, ...,    0,    0,    0]])>

In [199]:
embedding_dim=16

embedding_layer = layers.Embedding(
    max_features + 1, 
    embedding_dim,
    embeddings_initializer="uniform",
)


model = tf.keras.Sequential([
  embedding_layer,  
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(32, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(1)
])

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [200]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.5)])

In [201]:
epochs = 15
history = model.fit(
    x = text_np,
    y = labels,
    validation_split=0.2,
    epochs=epochs)

Epoch 1/15
5671/5671 ━━━━━━━━━━━━━━━━━━━━ 93s 16ms/step - binary_accuracy: 0.5897 - loss: 0.6387 - val_binary_accuracy: 0.7074 - val_loss: 0.8303
Epoch 2/15
5671/5671 ━━━━━━━━━━━━━━━━━━━━ 63s 11ms/step - binary_accuracy: 0.7058 - loss: 0.5345 - val_binary_accuracy: 0.8576 - val_loss: 0.6032
Epoch 3/15
5671/5671 ━━━━━━━━━━━━━━━━━━━━ 58s 10ms/step - binary_accuracy: 0.7385 - loss: 0.4907 - val_binary_accuracy: 0.7675 - val_loss: 0.7000
Epoch 4/15
5671/5671 ━━━━━━━━━━━━━━━━━━━━ 53s 9ms/step - binary_accuracy: 0.7511 - loss: 0.4763 - val_binary_accuracy: 0.8542 - val_loss: 0.5610
Epoch 5/15
5671/5671 ━━━━━━━━━━━━━━━━━━━━ 51s 9ms/step - binary_accuracy: 0.7580 - loss: 0.4638 - val_binary_accuracy: 0.6995 - val_loss: 0.7837
Epoch 6/15
5671/5671 ━━━━━━━━━━━━━━━━━━━━ 54s 9ms/step - binary_accuracy: 0.7601 - loss: 0.4611 - val_binary_accuracy: 0.5498 - val_loss: 1.0100
Epoch 7/15
5671/5671 ━━━━━━━━━━━━━━━━━━━━ 48s 8ms/step - binary_accuracy: 0.7651 - loss: 0.4559 - val_binary_accuracy: 0.6905 -

In [163]:
n=110003
print(raw_text_np[n])
print(labels[n])

rt у вас принято поздравлять родителей с такими датами 
1.0


In [164]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

export_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization_3            │ ?                      │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ ?                      │       160,033 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,033 (625.13 KB)

 Trainable params: 160,033 (625.13 KB)

 Non-trainable params: 0 (0.00 B)

In [206]:
export_model.predict(tf.data.Dataset.from_tensor_slices([[custom_standardization('После прочтения этой книги я чувствую себя счастливой живой и воодушевленной')]]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


array([[0.7098914]], dtype=float32)